ALL IMPORTS

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime


Use GPU

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")


✅ Using device: cuda


LSTM MODEL CLASS

In [4]:
class LSTMForecast(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1])
        return out

Forecast Return function (per ticker)

In [5]:
def forecast_return(ticker, start_date, end_date, window_size=30, forecast_horizon=30):
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)[["Close"]].dropna()
        df.columns = ["price"]
        
        mean = df['price'].mean()
        std = df['price'].std()
        scaled = (df['price'] - mean) / std
        data = scaled.values.astype(np.float32)

        sequences = []
        for i in range(len(data) - window_size - forecast_horizon):
            seq = data[i:i+window_size]
            label = data[i+window_size+forecast_horizon-1]
            sequences.append((seq, label))
        if len(sequences) == 0:
            return 0.0

        X = torch.tensor(np.array([s[0] for s in sequences])).unsqueeze(-1).to(device)
        y = torch.tensor(np.array([s[1] for s in sequences])).unsqueeze(-1).to(device)

        model = LSTMForecast().to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        for epoch in range(50):
            model.train()
            output = model(X)
            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        recent_seq = torch.tensor(data[-window_size:]).unsqueeze(0).unsqueeze(-1).to(device)
        pred_scaled = model(recent_seq).item()
        pred_price = pred_scaled * std + mean
        current_price = df["price"].iloc[-1]
        expected_return = (pred_price - current_price) / current_price
        return expected_return
    except:
        return 0.0

In [6]:
def get_optimal_portfolio(all_stocks, start_date, end_date, forecast_horizon=30, num_holdings=10):
    returns_dict = {}
    for stock in all_stocks:
        r = forecast_return(stock, start_date, end_date, forecast_horizon=forecast_horizon)
        returns_dict[stock] = r
        # print(f"{stock}: {r:.2%} expected return")

    # Sort and select top N
    top_stocks = sorted(returns_dict.items(), key=lambda x: x[1], reverse=True)[:num_holdings]
    selected_stocks = [s[0] for s in top_stocks]
    selected_returns = np.array([s[1] for s in top_stocks])

    # Get historical returns
    price_data = yf.download(selected_stocks, start=start_date, end=end_date, progress=False)["Close"]
    price_data = price_data.dropna(axis=1, how='all')
    returns = price_data.pct_change().dropna()
    cov_matrix = returns.cov().values * 252

    # Mean-variance optimization
    num = len(selected_stocks)
    init_weights = np.ones(num) / num
    bounds = [(0, 1) for _ in range(num)]
    constraints = {"type": "eq", "fun": lambda x: np.sum(x) - 1}

    def portfolio_risk(w, r, c):
        return np.sqrt(np.dot(w.T, np.dot(c, w)))

    opt = minimize(portfolio_risk, init_weights, args=(selected_returns, cov_matrix),
                   method='SLSQP', bounds=bounds, constraints=constraints)

    optimal_weights = opt.x

    print("\nOptimal Portfolio Allocation:")
    for s, w in zip(selected_stocks, optimal_weights):
        print(f"{s}: {w:.2%}")

    return selected_stocks, optimal_weights, selected_returns


Get High Volaility Stocks

In [7]:
def get_high_volatility_stocks(tickers, start_date, end_date, top_n=5):
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Close"]
    data = data.dropna(axis=1, how='all')
    daily_returns = data.pct_change().dropna()
    vol = daily_returns.std() * np.sqrt(252)  # Annualized volatility

    sorted_vol = vol.sort_values(ascending=False).head(top_n)
    
    print("\nTop High Volatility Stocks:")
    for ticker, v in sorted_vol.items():
        print(f"{ticker}: {v:.2%} annualized volatility")

    return sorted_vol.index.tolist()


Get S&p500

In [8]:
def get_sp500():
    tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    constituents = tables[0]
    constituents = constituents.rename(columns={"Symbol": "Ticker", "Security": "Company"})
    tickers = constituents["Ticker"].tolist()
    return tickers


In [10]:
top_n = 30
num_holdings = 5
forecast_horizon = 30

In [11]:
all_stocks = get_sp500()
start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

In [12]:
high_vol_stocks = get_high_volatility_stocks(all_stocks, start_date, end_date, top_n=top_n)
high_stocks, high_weights, high_returns = get_optimal_portfolio(
    high_vol_stocks, start_date, end_date, forecast_horizon=forecast_horizon, num_holdings=num_holdings)


YF.download() has changed argument auto_adjust default to True



2 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-03-29)')



Top High Volatility Stocks:
SMCI: 122.24% annualized volatility
VST: 76.76% annualized volatility
PLTR: 72.98% annualized volatility
TSLA: 71.84% annualized volatility
CEG: 71.47% annualized volatility
MRNA: 69.36% annualized volatility
WBA: 66.19% annualized volatility
INTC: 64.59% annualized volatility
AVGO: 62.81% annualized volatility
MPWR: 62.24% annualized volatility
ENPH: 61.70% annualized volatility
DXCM: 61.02% annualized volatility
WST: 60.40% annualized volatility
AXON: 60.22% annualized volatility
MU: 59.91% annualized volatility
NVDA: 57.76% annualized volatility
GEV: 57.35% annualized volatility
ALB: 53.79% annualized volatility
LW: 53.78% annualized volatility
ANET: 52.72% annualized volatility
WSM: 52.59% annualized volatility
WBD: 52.01% annualized volatility
FSLR: 51.95% annualized volatility
TER: 51.84% annualized volatility
ON: 50.92% annualized volatility
CRWD: 50.42% annualized volatility
LRCX: 49.30% annualized volatility
DLTR: 49.28% annualized volatility
TPL: 

In [ ]:
reg_stocks, reg_weights, reg_returns = get_optimal_portfolio(
    all_stocks, start_date, end_date, forecast_horizon=forecast_horizon, num_holdings=num_holdings)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

AAPL: 2.27% expected return



[*********************100%***********************]  1 of 1 completed

MSFT: 6.12% expected return



[*********************100%***********************]  1 of 1 completed

...

XOM: -7.60% expected return



[*********************100%***********************]  1 of 1 completed

PFE: 4.46% expected return



[*********************100%***********************]  5 of 5 completed

TGT: 11.53% expected return

Optimal Portfolio Allocation:
NVDA: 15.79%
TGT: 18.95%
GOOGL: 0.00%
MSFT: 47.29%
PFE: 17.97%


BACKTESTING!!!

Backtest High voltailty!!!

In [13]:
strategy_cutoff = "2025-01-01"
today = datetime.today().strftime("%Y-%m-%d")

high_vol_stocks = get_high_volatility_stocks(
    all_stocks, start_date, strategy_cutoff, top_n=top_n  # or any number you like
)
# Run portfolio selection using data *before* Jan 1, 2025
past_selected, past_weights, _ = get_optimal_portfolio(
    high_vol_stocks, start_date, strategy_cutoff, forecast_horizon=forecast_horizon, num_holdings=num_holdings
)

# Download price data from Jan 1, 2025 to today for backtest
future_prices = yf.download(past_selected, start=strategy_cutoff, end=today, progress=False)["Close"]
future_prices = future_prices.dropna(axis=1, how='all')
returns = future_prices.pct_change().dropna()

# Calculate portfolio value over time
weighted_returns = returns @ past_weights
cumulative_return = (1 + weighted_returns).cumprod()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(cumulative_return, label="Simulated Portfolio Value For High Voltailty")
plt.title("Backtest: Portfolio Performance Since Jan 2025")
plt.xlabel("Date")
plt.ylabel("Portfolio Value (Relative)")
plt.grid(True)
plt.legend()
plt.show()



182 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-01-01)')
['HOLX', 'HCA', 'RMD', 'IP', 'EQIX', 'EBAY', 'CBRE', 'BMY', 'PM', 'LMT', 'MKTX', 'SW', 'OTIS', 'LNT', 'MSCI', 'MA', 'FTV', 'COST', 'SNPS', 'DAY', 'HST', 'CCL', 'PTC', 'SJM', 'PPG', 'AMCR', 'TRGP', 'C', 'CRL', 'LVS', 'ROP', 'CEG', 'BLDR', 'CRM', 'AIG', 'VST', 'NWS', 'ECL', 'CVX', 'LLY', 'ODFL', 'CPAY', 'KDP', 'DGX', 'PCG', 'BAC', 'WST', 'EG', 'BK', 'LEN', 'BAX', 'MCD', 'ETR', 'KKR', 'POOL', 'ENPH', 'LW', 'CAH', 'AOS', 'PCAR', 'LOW', 'NKE', 'UHS', 'GL', 'GPC', 'WFC', 'PLTR', 'SCHW', 'LIN', 'STE', 'HPQ', 'ISRG', 'FITB', 'MAA', 'PYPL', 'DRI', 'REG', 'DAL', 'TXT', 'MNST', 'TRV', 'ZBH', 'MLM', 'CMS', 'LKQ', 'ESS', 'DOW', 'CF', 'PWR', 'CTVA', 'DHR', 'ABT', 'VTR', 'PPL', 'PANW', 'FFIV', 'WBD', 'SPGI', 'ED', 'HUM', 'TTWO', 'SOLV', 'JKHY', 'HD', 'KLAC', 'OKE', 'FOX', 'CPT', 'PNW', 'KEYS', 'MAR', 'ATO', '


Top High Volatility Stocks:
SMCI: 112.50% annualized volatility
TSLA: 67.84% annualized volatility
MRNA: 60.57% annualized volatility
DXCM: 59.53% annualized volatility
AVGO: 58.04% annualized volatility
WBA: 56.76% annualized volatility
FSLR: 56.13% annualized volatility
MPWR: 55.90% annualized volatility
ALB: 54.58% annualized volatility
DELL: 54.40% annualized volatility
INTC: 54.33% annualized volatility
MU: 53.06% annualized volatility
NVDA: 52.89% annualized volatility
WSM: 52.81% annualized volatility
TPL: 51.44% annualized volatility
PARA: 51.32% annualized volatility
NCLH: 48.73% annualized volatility
AXON: 48.07% annualized volatility
GEV: 47.86% annualized volatility
DG: 47.79% annualized volatility
LRCX: 46.26% annualized volatility
UAL: 45.96% annualized volatility
AMD: 45.91% annualized volatility
EL: 44.23% annualized volatility
EW: 43.91% annualized volatility
DLTR: 42.28% annualized volatility
MHK: 42.19% annualized volatility
MOH: 42.05% annualized volatility
SBUX: 4


1 Failed download:
['SMCI']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['MRNA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['DXCM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['AVGO']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['WBA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['MPWR']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['DELL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['MU']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['WSM']: YFRateLimitError('Too Many Requests. Rate limited. Try a

ValueError: shapes (4,4) and (5,) not aligned: 4 (dim 1) != 5 (dim 0)

General Backtesting

In [ ]:
strategy_cutoff = "2025-01-01"
today = datetime.today().strftime("%Y-%m-%d")


# Run portfolio selection using data *before* Jan 1, 2025
past_selected, past_weights, _ = get_optimal_portfolio(
    all_stocks, start_date, strategy_cutoff, forecast_horizon=forecast_horizon, num_holdings=num_holdings
)

# Download price data from Jan 1, 2025 to today for backtest
future_prices = yf.download(past_selected, start=strategy_cutoff, end=today, progress=False)["Close"]
future_prices = future_prices.dropna(axis=1, how='all')
returns = future_prices.pct_change().dropna()

# Calculate portfolio value over time
weighted_returns = returns @ past_weights
cumulative_return = (1 + weighted_returns).cumprod()

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(cumulative_return, label="Simulated Portfolio Value")
plt.title("Backtest: Portfolio Performance Since Jan 2025")
plt.xlabel("Date")
plt.ylabel("Portfolio Value (Relative)")
plt.grid(True)
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********